In [0]:
%pip install -qU langchain langchain-community langchain-core sqlalchemy langchain_experimental matplotlib mlflow langgraph fsspec huggingface_hub youtube-search openai langchain-databricks databricks-sdk langchain_openai

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython() 

In [0]:
# This script initializes a deployment client, creates a new endpoint, and configures it to serve a specific model version.
# It sets up the endpoint with traffic routing to direct 100% of the traffic to the specified model version and enables scaling to zero.

import mlflow
from mlflow.deployments import get_deploy_client

mlflow.set_registry_uri("databricks-uc")
client = get_deploy_client("databricks")

endpointname = "qaxxxx"
catalog = "llmcup2024"
schema = "default"
modelname = "qax"

endpoint = client.create_endpoint(
  name=f"{endpointname}",
  config={
    "served_entities": [
        {
            "entity_name": f"{catalog}.{schema}.{modelname}",
            "entity_version": "1",
            "workload_size": "Small",
            "scale_to_zero_enabled": True
        }
    ],
    "traffic_config": {
        "routes": [
            {
                "served_model_name": f"{modelname}-1",
                "traffic_percentage": 100
            }
        ]
    }
  }
)

In [0]:
# This script initializes a WorkspaceClient, lists all available serving endpoints, and prints their names.
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
available_endpoints = [x.name for x in w.serving_endpoints.list()]
display(available_endpoints)

['agentqax2', 'qaxtest', 'databricks-meta-llama-3-1-70b-instruct', 'databricks-meta-llama-3-1-405b-instruct', 'databricks-gte-large-en', 'databricks-dbrx-instruct', 'databricks-mixtral-8x7b-instruct', 'databricks-bge-large-en', 'databricks-llama-2-70b-chat']


In [0]:
# Import necessary modules from the databricks.sdk package
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import ChatMessage, ChatMessageRole

# Define the name of the serving endpoint to be queried
serving_endpoint_name = "agentqax2"

# Prepare the prompt and the message to be sent to the serving endpoint
prompt = "Answer this question like a helpful assistant: "
messages = [{"messages": [{"role": "user", "content": prompt + "provide feedbacks for Ali to improve greeting and closing skills"}]}]

# Query the serving endpoint with the prepared messages to get a response
response = w.serving_endpoints.query(
            name=serving_endpoint_name,
            inputs=messages,
        )

In [0]:
print("----------------input---------------")
print(messages[0]['messages'][0]["content"])
print("----------------output---------------")
print(response.predictions[-1]['messages'][-1]["content"])

----------------input---------------
Answer this question like a helpful assistant: provide feedbacks for Ali to improve greeting and closing skills
----------------output---------------
Here is a rewritten version of the response in a more concise and formal tone:

**Feedback and Training Plan for Ali**

**Summary:**
Ali demonstrates strong product knowledge and effectively resolves customer issues. However, there is room for improvement in greeting and closing interactions.

**Key Areas for Improvement:**

1. **Greeting Skills:**
	* Establish a rapport with customers through personalized greetings and open-ended questions.
	* Practice friendly and engaging greetings.
2. **Closing Skills:**
	* Develop a consistent closing script that includes thanking customers, summarizing the resolution, and providing clear next steps.
	* Emphasize ensuring customers feel valued and supported.

**Training Plan:**

1. Record and review greetings to identify areas for improvement.
2. Role-play closing

### Mlflow run model log
<img src ='./serving_endpoint.png'>